# Water Observations from Space (WOfS)
The [DEA water observation product](https://www.dea.ga.gov.au/products/dea-water-observation) maps the presense of surface water from Landsat imagery. An analysis through time can map the frequency a pixel is inundated by water, which can be used to infer the temporal and spatial statistics of flood/drought events. This notebook demonstrates how to run the DEA water observation algorithm for a given area of interest

- [DEA product and algorithm details](https://cmi.ga.gov.au/data-products/dea/613/dea-water-observations-landsat)
- [Reference code](https://github.com/GeoscienceAustralia/wofs/blob/master/wofs/virtualproduct.py)

This notebook requires the "WOfS Environment", which needs to be installed on the first run of this notebook by executing the following cell. Once done, set the environment in the top-right corner. 

![](../resources/kernel_wofs.png)

In [ ]:
# Install the WOfS environment
#  - Only installs the environment if required (run it at least once)
#  - Select the "WOfS Environment" kernel as per the picture above
!../tools/install_wofs.sh

# Table of contents
* [Data parameters - Lake Tempe](#Data-parameters---Lake-Tempe)
* [Set proxy parameters to access data locally](#Set-proxy-parameters-to-access-data-locally)
* [Import dependencies and initialise datacube](#Import-dependencies-and-initialise-datacube)
* [Display the region of interest](#Display-the-region-of-interest)
* [Load and display the DEM](#Load-and-display-the-DEM)
* [Load the data](#Load-the-data)
* [Classify WOfS](#Classify-WOfS)
* [Water Observations Summaries](#Water-Observations-Summaries)
  * [Wet counts](#Wet-counts)
  * [Clear observations counts](#Clear-observations-counts)
  * [Wet frequencies](#Wet-frequencies)


## Data parameters - Lake Tempe

In [ ]:
product = "landsat8_c2l2_sr"
longitude = (119.8242517, 120.0350519)
latitude = (-4.2013799, -3.9445384)
time = ('2020-01-01', '2020-12-31')
output_crs = "EPSG:32650"
resolution = (30, -30)

# Where to save the DEM fetched in ODC
DEM_PATH = "/home/jovyan/dems/srtm_lake_tempe.tif"

## Set proxy parameters to access data locally

In [ ]:
from os import environ

environ["AWS_HTTPS"] = "NO"
environ["GDAL_HTTP_PROXY"] = "easi-caching-proxy.caching-proxy:80"
print(f'Will use caching proxy at: {environ.get("GDAL_HTTP_PROXY")}')

## Import dependencies and initialise datacube

In [ ]:
import sys
sys.path.append("../../hub-notebooks/scripts")

from pathlib import Path

import xarray as xr
import rioxarray

from app_utils import display_map
from wofs.virtualproduct import WOfSClassifier

try:
    from dea_tools.plotting import display_map, rgb
except ImportError:
    # Local copy of selected dea_tools
    if 'tools/' not in sys.path:
        sys.path.append('tools/')
    from datacube_utils import display_map
    rgb = None  # Not copied or adapted yet

In [ ]:
from datacube import Datacube
from datacube.utils.aws import configure_s3_access

configure_s3_access(
    aws_unsigned=False, 
    requester_pays=True, 
);

dc = Datacube()

In [ ]:
# Display available products
products_info = dc.list_products()
products_info

## Display the region of interest

In [ ]:
display_map(x=longitude, y=latitude)

## Load and display the DEM
We load the SRTM DEM data from ODC using the lpdaac driver.
The Lake Tempe area is mostly flat so the DEM can be made optional.

In [ ]:
# Ignore warnings in output
import warnings
from sqlalchemy.exc import SAWarning
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=SAWarning) 

from os import environ
from cartopy.crs import PlateCarree
from datashader import reductions
import holoviews as hv
import hvplot.xarray
import matplotlib.pyplot as plt

dem = dc.load(
    product="lpdaac_nasadem", 
    latitude=latitude,
    longitude=longitude,
    output_crs="epsg:4326", 
    resolution=(-1/3600, 1/3600),
)
elevation = dem.elevation

In [ ]:
options = {
    'title': 'Elevation',
    'width': 800,
    'height': 500,
    'aspect': 'equal',
    'cmap': plt.cm.terrain,
    'clim': (0, elevation.max().values.item()),    # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}
plot_crs = PlateCarree()
elevation.hvplot.image(
     x = 'longitude', y = 'latitude',         # Dataset x,y dimension names 
     crs = plot_crs,
     rasterize = True,                        # If False, data will not be reduced. This is slow to load but all data is loaded.
     aggregator = reductions.mean(),          # Datashader calculates the mean value for reductions (also first, min, max, las, std, mode)
     precompute = True,                       # Datashader precomputes what it can
    ).opts(**options).hist(bin_range = options['clim'])

In [ ]:
dem_path = Path(DEM_PATH)
dem_path.parent.mkdir(parents=True, exist_ok=True)
elevation.rio.to_raster(dem_path)

## Load the data
Load the data from ODC and rename bands as needed by the WOfS classifier.

In [ ]:
# Ignore SAWarning in output
measurements = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']
data = dc.load(
    product=product,
    longitude=longitude,
    latitude=latitude,
    time=time,
    output_crs=output_crs,
    resolution=resolution,
    measurements=measurements,
    dask_chunks={'time': 1},
)

data = data.rename({
    "blue": "nbart_blue",
    "green": "nbart_green",
    "red": "nbart_red",
    "nir": "nbart_nir",
    "swir1": "nbart_swir_1",
    "swir2": "nbart_swir_2",
    "pixel_qa": "fmask",
})
data

## Classify WOfS

In [ ]:
transform = WOfSClassifier(c2_scaling=True, dsm_path=DEM_PATH)
# Compute the WOFS layer
wofl = transform.compute(data)
wofl

In [ ]:
# Uncomment the following line to display WOfS for each time slice. This may take a few minutes
# wofl.water.plot(col="time", col_wrap=5);

## Water observations summaries
The Water Observations Summaries based on [https://github.com/opendatacube/odc-stats/blob/develop/odc/stats/plugins/wofs.py](https://github.com/opendatacube/odc-stats/blob/develop/odc/stats/plugins/wofs.py) are made up of:
- `count_clear`: a count of every time a pixel was observed (not obscured by terrain or clouds)
- `count_wet`: a count of every time a pixel was observed and wet
- `frequency`: what fraction of time (wet/clear) was the pixel wet

In [ ]:
# Rename dimensions as required
wofl = wofl.rename({"x": "longitude", "y": "latitude"})

In [ ]:
from odc.algo import safe_div, apply_numexpr, keep_good_only

wofl["bad"] = (wofl.water & 0b0111_1110) > 0
wofl["some"] = apply_numexpr("((water<<30)>>30)==0", wofl, name="some")
wofl["dry"] = wofl.water == 0
wofl["wet"] = wofl.water == 128
wofl = wofl.drop_vars("water")
for dv in wofl.data_vars.values():
    dv.attrs.pop("nodata", None)

In [ ]:
# Ignore warnings triggered by time slices without data at all
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide") 
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide") 

wofl.wet.plot(col="time", col_wrap=5);

In [ ]:
# Helper frunction from https://github.com/opendatacube/odc-stats/blob/develop/odc/stats/plugins/wofs.py
def reduce(xx: xr.Dataset) -> xr.Dataset:
    nodata = -999
    count_some = xx.some.sum(axis=0, dtype="int16")
    count_wet = xx.wet.sum(axis=0, dtype="int16")
    count_dry = xx.dry.sum(axis=0, dtype="int16")
    count_clear = count_wet + count_dry
    frequency = safe_div(count_wet, count_clear, dtype="float32")

    count_wet.attrs["nodata"] = nodata
    count_clear.attrs["nodata"] = nodata

    is_ok = count_some > 0
    count_wet = keep_good_only(count_wet, is_ok)
    count_clear = keep_good_only(count_clear, is_ok)

    return xr.Dataset(
        dict(
            count_wet=count_wet,
            count_clear=count_clear,
            frequency=frequency,
        )
    )

In [ ]:
summary = reduce(wofl)

### Wet counts

A count of every time a pixel was observed and wet.

In [ ]:
summary.count_wet.plot(size=10)
plt.title("Lake Tempe – Wet observations counts for 2020");

### Clear observations counts

A count of every time a pixel was observed (not obscured by terrain or clouds).

In [ ]:
summary.count_clear.plot(size=10)
plt.title("Lake Tempe – Clear observations counts for 2020");

### Wet frequencies

What fraction of the time was the pixel wet.

In [ ]:
summary.frequency.plot(size=10)
plt.title("Lake Tempe – Wet frequency for 2020");

In [ ]:
# Save frequencies to high-res image file
summary.frequency.plot(size=20)
plt.title("Lake Tempe – Wet frequency for 2020")
plt.savefig('/home/jovyan/tempe_wofs_2020.png', dpi=600);